# Camera

Each drone is equipped with a single 5 megapixel Raspberry Pi Camera. The camera is used to measure motion in the planar
directions. This camera points down towards the ground to measure `x`, `y`,
and yaw velocities of the drone using optical flow vectors that are
extracted from the camera images. This is a lightweight task, meaning
that it does not require a lot of computational effort, because these vectors
are already calculated by the Raspberry Pi's image processor for h264 video
encoding. We also use the camera to estimate the relative position of
the drone by estimating the rigid transformations between two images.


# Estimating Velocity by Summing Optical Flow Vectors

We want to estimate our $x$ and $y$ velocity using the Duckiedrone's camera. Thankfully, optical flow from the Raspberry Pi Camera is calculated on board the Raspberry Pi itself. All we have to do is process the optical flow vectors that have already been calculated for us!

To calculate the $x$ velocity, we have to sum the $x$ components of all the optical flow vectors and multiply the sum by some normalization constant. We calculate the $y$ velocity in the same way.

Let $c$ be the normalization constant that allows us to convert the sum of components of optical flow vectors into a velocity.

How do we calculate $c$? Well, it must have something to do with the current height of the drone. Things that are far away move more slowly across your field of view. If a drone is at a height of $0.6 m$ and a feature passes through its camera's field of view in 1 second, then that drone is moving faster than another drone at a height of $0.1 m$ whose camera also passes over the same feature in 1 second. If we let $a$ be the altitude of the drone, then the drone's normalization constant must be $c = ab$, where $b$ is some number that accounts for the conversion of optical flow vectors multiplied by an altitude to a velocity. You do not have to worry about calculating $b$ (the *flow coefficient*), as it is taken care of for you.

In summary, to calculate the $x$ velocity, we have to sum the $x$ components of the optical flow vectors and then multiply the sum by $ab$. The $y$ velocity is calculated in the exact same way.

## Questions
1.  The Raspberry Pi calculates the following optical flow vectors: $[5, 4]$, $[1, 2]$, and $[3, 2]$. The flow vectors are in the form [$x$-component, $y$-component]. What are your $x$ and $y$ velocities $\dot{x}$ and $\dot{y}$? Your solution will be in terms of $a$, the altitude, and $b$, the flow coefficient.

# Velocity Estimation via Optical Flow

In this part of the project you will create a class that interfaces with the Raspberry Pi Camera to extract planar velocities from optical flow vectors.

## Code Structure
To interface with the camera, you will be using the `raspicam_node` library. This library publishes both images and optical flow vectors to ROS topics.  You will estimate velocity using the flow vectors, and estimate small changes in position by extracting features from pairs of frames. In the sensors project repo, we've included a script called `student_optical_flow.py` which you will edit to have it publish the estimated velocity from the flow vectors.

Similarly a second script is `student_rigid_transform.py` which you will edit, to have it subscribe to the image topic and publish position estimates. 

## Analyze and Publish the Sensor Data
On your drones, the chip on the Raspberry Pi dedicated to video processing from the camera calculates motion vectors ([optical flow](https://en.wikipedia.org/wiki/Optical_flow)) automatically for H.264 video encoding. [Click here to learn more](https://www.raspberrypi.org/blog/vectors-from-coarse-motion-estimation/). You will be analyzing these motion vectors in order to estimate the velocity of your drone.

# Exercises

You will now implement your velocity estimation using optical flow by completing all of the `TODO`'s in `student_optical_flow.py`. There are two methods you will be implementing.

The first method is `setup`, which will be called to initialize the instance variables.

  1. Create a ROS publisher to publish the velocity values. 
  
  2. The perspicacious roboticist may have noticed that magnitude of the velocity in global coordinates is dependent on the height of the drone. Add a subscriber to the topic `/pidrone/state` to your `AnalyzeFlow` class and save the z position value to a class variable in the callback. Use this variable to scale the velocity measurements by the height of the drone (the distance the camera is from what it is perceiving).

The second method is `motion_cb`, which is called every time that the camera gets a set of flow vectors, and is used to analyze the flow vectors to estimate the x and y velocities of your drone.

  1. Estimate the velocities, using the `TODO`'s as a guide.

  2. Publish the velocities.

## Check your Measurements
You'll want to make sure that the values you're publishing make sense. To do this, you'll be echoing the values that you're publishing and empirically verifying that they are reasonable.

1. Start up your drone and launch a screen
2. Navigate to \`4 and quit the node that is running
3. Run `python ~/catkin_ws/src/project-sensors-implementation-<your-GitHub-username>/student_optical_flow_node.py`
4. Enter `rostopic echo /pidrone/picamera/twist`
5. Move the drone by hand to the left and right and forward and backward to verify that the measurements make sense
6. Verify that the velocity values are reasonable (roughly in the range of $-1m/s$ to $1m/s$) and have the correct sign (positive when the drone is moving to the right or up, and negative to the left or down).

